# 🏭 THE CONVERSATION FACTORY V5: High-Quality Synthetic Data (Regex Fix)

**Mission:** Turn raw Indonesian Laws into high-quality QA pairs for Nuzantara.
**Updates in V5:**
- **Fix:** Correctly separates the Prompt from the Response before searching for JSON.
- **Regex:** Finds the JSON object strictly within the generated answer.
**Model:** `Llama-3-8B-Instruct` (via Unsloth) - Running on Colab GPU.
**Output:** `synthetic_gold_dataset.json`.

## 🚀 Setup
We use Colab T4 or A100 GPU.

In [ ]:
# 🚀 Install Dependencies (Unsloth for fast inference)
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install langchain langchain-community pypdf tiktoken

In [ ]:
# 📥 DOWNLOAD DATA (Direct from Drive)
!gdown 1Lx4y9TQ45uBUyvNzeHiHinxo_k_WOMmm -O /content/nuzantara_laws.zip

# 📦 Unzip
import os
import zipfile

ZIP_PATH = "/content/nuzantara_laws.zip"
EXTRACT_DIR = "/content/nuzantara_laws"

if os.path.exists(ZIP_PATH):
    print("📂 Extracting...")
    with zipfile.ZipFile(ZIP_PATH, "r") as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print("✅ Extraction complete!")
    SOURCE_DIR = EXTRACT_DIR
else:
    print("❌ Zip file not found!")

In [ ]:
# 🧠 CORE LOGIC INJECTION (The Butcher System V3)
# Includes Smart Fallback for non-standard documents

import re
from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- CONSTANTS ---
NOISE_PATTERNS = [
    re.compile(r"^Halaman\s+\d+\s+dari\s+\d+", re.IGNORECASE | re.MULTILINE),
    re.compile(
        r"^Salinan sesuai dengan aslinya.*?(?=\n)",
        re.IGNORECASE | re.MULTILINE | re.DOTALL,
    ),
    re.compile(r"^PRESIDEN REPUBLIK INDONESIA\s*\n", re.IGNORECASE | re.MULTILINE),
    re.compile(r"^\s*-\s*\d+\s*-\s*$", re.MULTILINE),
    re.compile(r"\n{3,}", re.MULTILINE),
    re.compile(r"^\s*\d+\s*$", re.MULTILINE),
]

LEGAL_TYPE_PATTERN = re.compile(
    r"(UNDANG-UNDANG|PERATURAN PEMERINTAH|KEPUTUSAN PRESIDEN|PERATURAN MENTERI|QANUN|PERATURAN DAERAH|PERATURAN KEPALA)",
    re.IGNORECASE,
)

LEGAL_TYPE_ABBREV = {
    "UNDANG-UNDANG": "UU",
    "PERATURAN PEMERINTAH": "PP",
    "KEPUTUSAN PRESIDEN": "Keppres",
    "PERATURAN MENTERI": "Permen",
    "QANUN": "Qanun",
    "PERATURAN DAERAH": "Perda",
    "PERATURAN KEPALA": "Perkep",
}

NUMBER_PATTERN = re.compile(r"NOMOR\s+(\d+[A-Z]?)(?:[/-]\d+)?", re.IGNORECASE)
YEAR_PATTERN = re.compile(r"TAHUN\s+(\d{4})", re.IGNORECASE)
TOPIC_PATTERN = re.compile(
    r"TENTANG\s+(.+?)(?=DENGAN RAHMAT|Menimbang|Mengingat|$)", re.IGNORECASE | re.DOTALL
)

PASAL_PATTERN = re.compile(
    r"^Pasal\s+(\d+[A-Z]?)\s*(.+?)(?=^Pasal\s+\d+|^BAB\s+|^Penjelasan|\Z)",
    re.IGNORECASE | re.MULTILINE | re.DOTALL,
)
BAB_PATTERN = re.compile(
    r"^BAB\s+([IVX]+|[A-Z]+|\d+)\s+(.+?)(?=\n|$)", re.IGNORECASE | re.MULTILINE
)
AYAT_PATTERN = re.compile(
    r"(?:^|\n)\s*\((\d+)\)\s*(.+?)(?=(?:^|\n)\s*\(\d+\)|$)", re.MULTILINE | re.DOTALL
)

# --- CLASSES ---


class LegalCleaner:
    def clean(self, text: str) -> str:
        if not text:
            return ""
        cleaned = text
        for pattern in NOISE_PATTERNS:
            cleaned = pattern.sub("", cleaned)
        cleaned = re.sub(r"\s+", " ", cleaned)
        cleaned = re.sub(r"\n\s+\n", "\n\n", cleaned)
        cleaned = re.sub(
            r"Pasal\s+(\d+[A-Z]?)", r"Pasal \1", cleaned, flags=re.IGNORECASE
        )
        return cleaned.strip()


class LegalMetadataExtractor:
    def extract(self, text: str) -> dict:
        meta = {
            "type": "UNKNOWN",
            "number": "UNKNOWN",
            "year": "UNKNOWN",
            "topic": "UNKNOWN",
        }

        type_match = LEGAL_TYPE_PATTERN.search(text)
        if type_match:
            doc_type = type_match.group(1).upper()
            meta["type"] = doc_type
            meta["type_abbrev"] = LEGAL_TYPE_ABBREV.get(doc_type, doc_type)

        num_match = NUMBER_PATTERN.search(text)
        if num_match:
            meta["number"] = num_match.group(1)

        year_match = YEAR_PATTERN.search(text)
        if year_match:
            meta["year"] = year_match.group(1)

        topic_match = TOPIC_PATTERN.search(text)
        if topic_match:
            meta["topic"] = re.sub(r"\s+", " ", topic_match.group(1).strip())[:200]

        return meta


class LegalStructureParser:
    def parse(self, text: str) -> dict:
        structure = {"batang_tubuh": []}
        bab_matches = list(BAB_PATTERN.finditer(text))
        for i, match in enumerate(bab_matches):
            bab_num = match.group(1)
            bab_title = match.group(2).strip()
            start = match.end()
            end = bab_matches[i + 1].start() if i + 1 < len(bab_matches) else len(text)

            # Simple pasal extraction for context finding
            pasal_list = []
            pasal_matches = list(PASAL_PATTERN.finditer(text[start:end]))
            for p_match in pasal_matches:
                pasal_list.append({"number": p_match.group(1)})

            structure["batang_tubuh"].append(
                {"number": bab_num, "title": bab_title, "pasal": pasal_list}
            )
        return structure


class LegalChunker:
    def __init__(self, max_pasal_tokens=1000):
        self.max_pasal_tokens = max_pasal_tokens
        self.fallback_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000, chunk_overlap=200
        )

    def chunk(self, text: str, metadata: dict, structure: dict) -> list:
        chunks = []
        pasal_matches = list(PASAL_PATTERN.finditer(text))

        # If no pasal found, use Fallback Splitter instead of sending whole text
        if not pasal_matches:
            raw_chunks = self.fallback_splitter.split_text(text)
            context = self._build_context(metadata)
            for rc in raw_chunks:
                chunks.append(self._create_chunk(rc, context, metadata))
            return chunks

        for match in pasal_matches:
            pasal_num = match.group(1)
            pasal_text = match.group(2).strip()

            # Find BAB context
            bab_context = self._find_bab_for_pasal(structure, pasal_num)

            # Check length - split by Ayat if needed
            if len(pasal_text) > 3000:  # Approx chars for 1000 tokens
                ayat_matches = list(AYAT_PATTERN.finditer(pasal_text))
                if ayat_matches:
                    for am in ayat_matches:
                        ayat_num = am.group(1)
                        ayat_text = am.group(2).strip()
                        context = self._build_context(
                            metadata, bab_context, f"Pasal {pasal_num}"
                        )
                        chunks.append(
                            self._create_chunk(
                                f"Ayat ({ayat_num})\n{ayat_text}",
                                context,
                                metadata,
                                pasal_num,
                            )
                        )
                    continue

            # Default: Pasal as chunk
            context = self._build_context(metadata, bab_context, f"Pasal {pasal_num}")
            chunks.append(self._create_chunk(pasal_text, context, metadata, pasal_num))

        return chunks

    def _build_context(self, meta, bab=None, pasal=None):
        parts = [
            meta.get("type_abbrev", "UNK"),
            f"NO {meta.get('number', '?')}",
            f"TAHUN {meta.get('year', '?')}",
            f"TENTANG {meta.get('topic', 'UNK')}",
        ]
        if bab:
            parts.append(bab)
        if pasal:
            parts.append(pasal)
        return f"[CONTEXT: {' - '.join(parts)}]"

    def _create_chunk(self, content, context, meta, pasal_num=None):
        chunk_text = f"{context}\n\n{content}"
        c = {"text": chunk_text, "has_context": True}
        c.update(meta)
        if pasal_num:
            c["pasal_number"] = pasal_num
        return c

    def _find_bab_for_pasal(self, structure, pasal_num):
        for bab in structure.get("batang_tubuh", []):
            for p in bab.get("pasal", []):
                if p.get("number") == pasal_num:
                    return f"BAB {bab.get('number')} - {bab.get('title', '')}"
        return None


print("✅ Nuzantara Legal Engine Loaded Successfully!")

In [ ]:
# 🦙 Load Llama 3 (4-bit Quantized)
from unsloth import FastLanguageModel

max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(model)

print("✅ Llama 3 Loaded! Ready to generate.")

In [ ]:
# 🎭 The Prompt Template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert Indonesian Legal AI trainer. Your task is to generate a synthetic QA pair based on the provided law text.
1. **User Question**: Create a realistic question a business owner or foreigner might ask about this specific article. It can be slightly informal.
2. **Ideal Answer**: Write the perfect, legally accurate answer based ONLY on the provided text. Cite the specific Article (Pasal) if mentioned.
3. **Category**: Classify as 'Visa', 'Tax', 'Corporate', or 'General'.

Output valid JSON format: {{"question": "...", "answer": "...", "category": "..."}}

### Input:
{}

### Response:
"""


def generate_qa(text_chunk):
    if len(text_chunk) > 10000:
        text_chunk = text_chunk[:10000] + "..."

    inputs = tokenizer([alpaca_prompt.format(text_chunk)], return_tensors="pt").to(
        "cuda"
    )

    outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
    response = tokenizer.batch_decode(outputs)[0]

    # ROBUST JSON EXTRACTION
    try:
        # 1. Split to get only the response part (avoid matching prompt examples)
        if "### Response:" in response:
            generated_text = response.split("### Response:")[-1]
        else:
            generated_text = response

        # 2. Find the first '{' and the last '}' in the GENERATED text
        match = re.search(r"\{.*\}", generated_text, re.DOTALL)
        if match:
            json_str = match.group(0)
            return json.loads(json_str)
        else:
            # Fallback: maybe it didn't output JSON at all
            print(f"⚠️ No JSON found in generated text. Raw: {generated_text[:100]}...")
            return None
    except Exception:
        # print(f"❌ JSON Parse Error: {e}. Raw: {response[:100]}...") # Too noisy
        return None

In [ ]:
# 🏭 Production Line (The Factory)
import glob
import random
import json
from tqdm.notebook import tqdm
from langchain_community.document_loaders import PyPDFLoader

# Initialize Engine
cleaner = LegalCleaner()
extractor = LegalMetadataExtractor()
parser = LegalStructureParser()
chunker = LegalChunker()

files = glob.glob(f"{SOURCE_DIR}/**/*.pdf", recursive=True)
dataset = []

print(f"📚 Found {len(files)} PDF files. Starting generation...")

# Sample random files for demo (remove slicing for full run)
TARGET_SAMPLES = 50  # How many QA pairs to generate for this test run
processed_count = 0

# Shuffle files to get variety
random.shuffle(files)

for file_path in tqdm(files, desc="Processing PDFs"):
    if processed_count >= TARGET_SAMPLES:
        break

    try:
        # Check PDF Header
        with open(file_path, "rb") as f:
            if f.read(4) != b"%PDF":
                continue

        # Load & Process
        loader = PyPDFLoader(file_path)
        pages = loader.load()
        raw_text = "\n".join([p.page_content for p in pages])

        if len(raw_text) < 100:
            continue

        cleaned_text = cleaner.clean(raw_text)
        meta = extractor.extract(cleaned_text)
        structure = parser.parse(cleaned_text)
        chunks = chunker.chunk(cleaned_text, meta, structure)

        # Pick 1 random chunk from this file to generate a QA pair
        if chunks:
            selected_chunk = random.choice(chunks)
            qa_pair = generate_qa(selected_chunk["text"])

            if qa_pair:
                qa_pair["source_file"] = os.path.basename(file_path)
                qa_pair["context_used"] = selected_chunk["text"]
                dataset.append(qa_pair)
                processed_count += 1
                print(
                    f"✅ Generated ({processed_count}/{TARGET_SAMPLES}): {qa_pair['question'][:50]}..."
                )

    except Exception:
        continue

# Save Dataset
OUTPUT_FILE = "/content/synthetic_gold_dataset.json"
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(dataset, f, indent=2)

print(f"🎉 Generation Complete! Saved {len(dataset)} pairs to {OUTPUT_FILE}")
print("⬇️ Download the file from the Files pane on the left!")